In [5]:
import pandas as pd


In [7]:
!pip install wikipedia
!pip install requests
!pip install BeautilfulSoup
!pip install time


  Could not find a version that satisfies the requirement BeautilfulSoup (from versions: )
No matching distribution found for BeautilfulSoup


  Could not find a version that satisfies the requirement time (from versions: )
No matching distribution found for time


In [1]:
!pip install lxml

In [1]:
import requests
from bs4 import BeautifulSoup
import bs4
import time
import numpy as np
import re
import time
#the goal is scrapping wikipedia in order to obtain information about citrus fruit
#We take the description section of each one and then we will extract main info.


In [74]:
page_citrus = 'https://en.wikipedia.org/wiki/List_of_citrus_fruits'
page_salads = 'https://en.wikipedia.org/wiki/List_of_leaf_vegetables'
page_spices = 'https://en.wikipedia.org/wiki/List_of_culinary_herbs_and_spices'

#A function that gets the URL of the page to be scraped
#,gets the html content and uses BeautifulSoup to parse html content

def make_soup(link):
    get_page = requests.get(link)
    html = get_page.content
    soup = BeautifulSoup(html, 'html.parser')
    return  soup


#####This function create a list with all the link of the foods in a wikipedia Page.
def make_link_list(wiki_page_to_scrap):
    start_time = time.time()
    link_table = []
    soup = make_soup(wiki_page_to_scrap)
    table = soup.find('table',{'class':'wikitable'})
    
            ### This first loop is used to scrap Wiki table Data.
    if isinstance(table , bs4.element.Tag):        
    
        table_cells = []
        table = soup.find('table',{'class':'wikitable'})
        for row in table.find_all("tr"):
            cells = row.find_all(['th' , 'td'])
            table_cells.append(cells)
        
        ### This loop is used to locate the "Common name" column index in our table cells
        indices = []
        for j in table_cells:
            for i, elem in enumerate(j):
                elem = str(elem)
                if 'name' in elem:
                    indices.append(i)
        indice = indices[0]
        print("indice table" , indices)
        
        #print('Table Cells' , table_cells[:3])
        ### Here we implement a loop to keep only the string of the Common name column.
        
        for cell in table_cells[2:]:
            print('len cell' , len(cell))
            print('cells' , cell)
            if (len(cell) < indice) == True :
                pass
             #   tmp_indice = indice - 1
              #  link_table.append(cell[tmp_indice].text)
            else:    
                print('Selected cell length' , len(cell) , cell[indice])
                link_table.append(cell[indice].text)
            
        ### Let's clean the table.
        for link in range(len(link_table)):
            link_table[link] = link_table[link].strip('\n')
            
        print('cpu time for the table schema = {:.4f} sec.'.format(time.time() - start_time))
            
            ### Here is when the Wiki page is just an Alphabetical List.    
    else:       
        for row in soup.find_all('div' , {'class':'div-col'}):
            for col in row.find_all('li'):
                species = col.text
                ###We just keep the common name of the species, because only the common name is used in recipes.
                only_common_species = re.sub("[\(\[].*?[\)\]]", "", species) 
                only_common_species , sep , tail = only_common_species.partition(',')
                link_table.append(only_common_species)
                
                ###Cleaning of the list, we remove all the occurence of string begining by List.
        for word in link_table[:]:
            if word.find('List') != -1:
                link_table.remove(word)
            
        print('cpu time for the list schema = {:.4f} sec.'.format(time.time() - start_time))     

    return link_table  

page_seeds = 'https://en.wikipedia.org/wiki/List_of_edible_seeds'


list_seeds = make_link_list(page_seeds)
print(list_seeds)
    

indice table [4]
len cell 6
cells [<td rowspan="32"><a href="/wiki/Poaceae" title="Poaceae">Poaceae</a>
</td>, <td rowspan="2"><a href="/wiki/Andropogoneae" title="Andropogoneae">Andropogoneae</a>
</td>, <td><i><a href="/wiki/Sorghum" title="Sorghum">Sorghum</a></i>
</td>, <td><a href="/wiki/Sorghum_bicolor" title="Sorghum bicolor"><i>S. bicolor</i></a>
</td>, <td>sorghum
</td>, <td><div class="center"><div class="floatnone"><a class="image" href="/wiki/File:Lalibela-Sorgho.jpg"><img alt="Lalibela-Sorgho.jpg" data-file-height="3240" data-file-width="4320" decoding="async" height="40" src="//upload.wikimedia.org/wikipedia/commons/thumb/5/57/Lalibela-Sorgho.jpg/53px-Lalibela-Sorgho.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/5/57/Lalibela-Sorgho.jpg/80px-Lalibela-Sorgho.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/5/57/Lalibela-Sorgho.jpg/107px-Lalibela-Sorgho.jpg 2x" width="53"/></a></div></div>
</td>]
Selected cell length 6 <td>sorghum
</td>
len cell 4
cells

IndexError: list index out of range

In [40]:
list_citrus = make_link_list(page_citrus)
print('Number of citrus' , len(list_citrus))
print(list_citrus[:3])

list_salad = make_link_list(page_salads) 
print('Number of salads' , len(list_salad))
print(list_salad[:10])

list_spices = make_link_list(page_spices)
print('Number of spices' , len(list_spices))


0
cpu time for the table schema = 0.2502 sec.
Number of citrus 51
['Amanatsu', 'Balady citronIsrael citron', 'Bergamot orange']
1
cpu time for the table schema = 0.7436 sec.
Number of salads 439
['China Jute', 'Climbing wattle', 'Paracress', 'Common Marshmallow', 'Purple amaranth', 'Common amaranth', 'Prickly amaranth', 'Amaranth', 'Slender amaranth', 'Celery']
cpu time for the list schema = 0.3598 sec.
Number of spices 203


In [15]:
# Let's create a dictionary with the different categories of vegetables. 

foods_dict = dict([('CITRUS' , list_citrus) , ('SALAD' , list_salad) , ('SPICES' , list_spices)])



In [43]:
page_seeds = 'https://en.wikipedia.org/wiki/List_of_edible_seeds'


list_seeds = make_link_list(page_seeds)
print(list_seeds)

4
<th>Tribe
</th>
<td rowspan="2"><a href="/wiki/Andropogoneae" title="Andropogoneae">Andropogoneae</a>
</td>
<td><i><a class="mw-redirect" href="/wiki/Zea_mays" title="Zea mays">Z. mays</a></i>
</td>


IndexError: list index out of range